##  **Advanced dMRI Feature Extraction** 
## CSD + Probabilistic Tractography + ROI-Based Tractometry with DIPY

This pipeline implements a **reproducible, atlas-free framework** for extracting microstructural and geometric features from diffusion MRI data, suitable for:

- Clinical biomarker discovery  
- Machine learning in neurodegenerative disease  
- Group-level white matter analysis  

### Methods Summary
- **Preprocessing**: Patch2Self denoising + Gibbs ringing removal  
- **Reconstruction**: Constrained Spherical Deconvolution (CSD, `sh_order=6`)  
- **Tractography**: Probabilistic Local Tracking with FA-based stopping  
- **Bundle Extraction**: Anatomically informed ROIs in native space  
- **Feature Extraction**: Along-tract profiles + geometric descriptors  

### Data & Software
- **Dataset**: Stanford HARDI (single-shell, *b* = 2000 s/mm², 150 diffusion directions)  
- **Software**: DIPY v1.7+, Python 3.9+

In [1]:
# 📦 Imports & Setup

import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # Suppress DIPY deprecation warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time

# Core libraries
from dipy.data import fetch_stanford_hardi, read_stanford_hardi
from dipy.core.gradients import gradient_table
import nibabel as nib

# Preprocessing
from dipy.denoise.patch2self import patch2self
from dipy.denoise.gibbs import gibbs_removal
from dipy.segment.mask import median_otsu

# Reconstruction
from dipy.reconst.csdeconv import auto_response_ssst, ConstrainedSphericalDeconvModel
from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity

# Tractography
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion
from dipy.tracking.streamline import Streamlines, select_by_rois
from dipy.tracking.utils import random_seeds_from_mask, length
from dipy.direction import ProbabilisticDirectionGetter

# Analysis
from dipy.stats.analysis import afq_profile
from dipy.data import get_sphere

# Configuration
OUTPUT_DIR = Path("./publication_dwi_features")
OUTPUT_DIR.mkdir(exist_ok=True)
plt.rcParams.update({"font.size": 10, "font.family": "sans-serif"})

print("✅ Imports and setup complete.")
print(f"📁 Output directory: {OUTPUT_DIR.resolve()}")


# 📥 1. Data Loading & Quality Control

print("=" * 80)
print("STEP 1: DATA LOADING & QUALITY CONTROL")
print("=" * 80)

# Load Stanford HARDI dataset
fetch_stanford_hardi()
hardi_img, gtab = read_stanford_hardi()
data = hardi_img.get_fdata()
affine = hardi_img.affine
voxel_size = hardi_img.header.get_zooms()[:3]

# Validate acquisition parameters
print(f"✓ Data shape: {data.shape} (x, y, z, volumes)")
print(f"✓ Voxel size: ({voxel_size[0]:.1f}, {voxel_size[1]:.1f}, {voxel_size[2]:.1f}) mm")
print(f"✓ b-values: {np.unique(gtab.bvals)} s/mm²")
print(f"✓ b=0 volumes: {np.sum(gtab.b0s_mask)}")
print(f"✓ Diffusion volumes: {np.sum(~gtab.b0s_mask)}")

# Signal quality metrics
b0_mean = np.mean(data[..., gtab.b0s_mask])
dwi_mean = np.mean(data[..., ~gtab.b0s_mask])
snr_estimate = b0_mean / np.std(data[data > 0])

print(f"\n📊 Signal Quality:")
print(f"  - Mean b=0 intensity: {b0_mean:.1f}")
print(f"  - Mean DWI intensity: {dwi_mean:.1f}")
print(f"  - Estimated SNR: {snr_estimate:.2f}")


# 🧹 2. Preprocessing

print("\n" + "=" * 80)
print("STEP 2: PREPROCESSING")
print("=" * 80)

# Patch2Self denoising
start = time.time()
print("\n🔧 Applying Patch2Self denoising...")
denoised_data = patch2self(data, gtab.bvals, model="ols", verbose=False)
print(f"  ✓ Completed in {time.time() - start:.1f}s")

# Gibbs ringing removal
print("\n🔧 Removing Gibbs ringing artifacts...")
denoised_data = gibbs_removal(denoised_data, slice_axis=2)
print("  ✓ Gibbs removal applied")

# Brain extraction
print("\n🧠 Extracting brain mask...")
b0_volumes = denoised_data[..., gtab.b0s_mask]
_, brain_mask = median_otsu(
    b0_volumes,
    vol_idx=range(min(5, b0_volumes.shape[-1])),
    median_radius=4,
    numpass=4,
    autocrop=False,
    dilate=2,
)
masked_data = denoised_data * brain_mask[..., None]

brain_voxels = np.sum(brain_mask)
print(f"  ✓ Brain voxels: {brain_voxels:,} ({100 * brain_voxels / brain_mask.size:.1f}%)")


# 🧬 3. Microstructural Reconstruction

print("\n" + "=" * 80)
print("STEP 3: MICROSTRUCTURAL RECONSTRUCTION")
print("=" * 80)

# DTI for FA-based stopping criterion
print("🔧 Fitting Diffusion Tensor Model...")
dti_model = TensorModel(gtab)
dti_fit = dti_model.fit(masked_data, mask=brain_mask)
FA = np.clip(np.nan_to_num(fractional_anisotropy(dti_fit.evals)), 0, 1)
MD = np.nan_to_num(mean_diffusivity(dti_fit.evals))
wm_mask = (FA > 0.2) & brain_mask

print(f"\n📊 White Matter Statistics (FA > 0.2):")
print(f"  FA:  {np.mean(FA[wm_mask]):.3f} ± {np.std(FA[wm_mask]):.3f}")
print(f"  MD:  {np.mean(MD[wm_mask]):.6f} mm²/s")

# CSD for fiber orientation estimation
print("\n🔧 Estimating CSD response function...")
response, ratio = auto_response_ssst(gtab, masked_data, roi_radii=10, fa_thr=0.7)
print(f"  ✓ Response eigenvalues: {response[0]}")
print(f"  ✓ FA threshold ratio: {ratio:.3f}")

print("\n🔧 Fitting CSD model (sh_order=6)...")
sphere = get_sphere("repulsion724")
csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order_max=6)
csd_fit = csd_model.fit(masked_data, mask=brain_mask)
GFA = np.nan_to_num(csd_fit.gfa)

print(f"\n📊 Generalized FA (GFA) in WM:")
print(f"  Mean: {np.mean(GFA[wm_mask]):.3f} ± {np.std(GFA[wm_mask]):.3f}")


# 🎯 4. Whole-Brain Tractography

print("\n" + "=" * 80)
print("STEP 4: WHOLE-BRAIN TRACTOGRAPHY")
print("=" * 80)

# Probabilistic direction getter
prob_dg = ProbabilisticDirectionGetter.from_shcoeff(
    csd_fit.shm_coeff,
    max_angle=30.0,
    sphere=sphere,
    relative_peak_threshold=0.25,
)

# Seeding and tracking
seeds = random_seeds_from_mask(wm_mask, affine=affine, seeds_count=2, seed_count_per_voxel=True)
print(f"🌱 Generated {len(seeds):,} seeds in white matter")

stopping_criterion = ThresholdStoppingCriterion(FA, threshold=0.15)
print("\n🧵 Running probabilistic tractography...")
start = time.time()
streamlines = Streamlines(LocalTracking(
    direction_getter=prob_dg,
    stopping_criterion=stopping_criterion,
    seeds=seeds,
    affine=affine,
    step_size=0.5,
    maxlen=1000,
    minlen=20,
))
print(f"✓ Generated {len(streamlines):,} streamlines in {time.time() - start:.1f}s")

# Length filtering
lengths = np.array(list(length(streamlines)))
valid = (lengths >= 20) & (lengths <= 200)
streamlines = streamlines[valid]
print(f"✓ Retained {len(streamlines):,} streamlines (20–200 mm)")


# 🌐 5. ROI-Based Bundle Extraction

print("\n" + "=" * 80)
print("STEP 5: ROI-BASED BUNDLE EXTRACTION")
print("=" * 80)

# Anatomical ROI definitions (in voxel space)
x_dim, y_dim, z_dim = data.shape[:3]
mid_x, mid_y, mid_z = x_dim // 2, y_dim // 2, z_dim // 2

bundle_rois = {
    "CC_ForcepsMajor": (mid_x-3, mid_x+4, mid_y-15, mid_y, mid_z-5, mid_z+5),      # Posterior corpus callosum
    "CC_ForcepsMinor": (mid_x-3, mid_x+4, mid_y+5, mid_y+20, mid_z-5, mid_z+5),     # Anterior corpus callosum
    "CST_L": (mid_x+5, mid_x+15, mid_y-10, mid_y+10, mid_z-15, mid_z+15),          # Left corticospinal tract
    "CST_R": (mid_x-15, mid_x-5, mid_y-10, mid_y+10, mid_z-15, mid_z+15),          # Right corticospinal tract
}

recognized_bundles = {}
for name, (x1, x2, y1, y2, z1, z2) in bundle_rois.items():
    roi = np.zeros_like(brain_mask)
    roi[x1:x2, y1:y2, z1:z2] = 1
    
    selected = list(select_by_rois(
        streamlines=streamlines,
        rois=[roi],
        affine=affine,
        include=[True],
        mode="any"
    ))
    
    if len(selected) > 10:
        recognized_bundles[name] = Streamlines(selected)
        print(f"  ✓ {name}: {len(selected)} streamlines")
    else:
        print(f"  ⚠️ {name}: insufficient streamlines ({len(selected)})")

print(f"\n✅ Successfully extracted {len(recognized_bundles)} bundles")


# 📊 6. Feature Extraction (Clinically Meaningful Only)

print("\n" + "=" * 80)
print("STEP 6: EXTRACTING ROBUST, CLINICALLY RELEVANT FEATURES")
print("=" * 80)

# Define bundles of interest
bundles_of_interest = {
    "CC_ForcepsMajor": recognized_bundles.get("CC_ForcepsMajor"),
    "CC_ForcepsMinor": recognized_bundles.get("CC_ForcepsMinor"),
    "CST_L": recognized_bundles.get("CST_L"),
    "CST_R": recognized_bundles.get("CST_R"),
}

# Global features (keep only microstructural)
global_features = {
    "subject_id": "stanford_hardi_001",
    "global_mean_FA": float(np.mean(FA[wm_mask])),
    "global_mean_MD": float(np.mean(MD[wm_mask])),
    "global_mean_GFA": float(np.mean(GFA[wm_mask])),
}

# Bundle-specific features (only FA, MD, GFA, asymmetry, quartiles)
for name, bundle in bundles_of_interest.items():
    if bundle is None or len(bundle) < 10:
        # If bundle missing, fill with NaNs
        prefix = name
        global_features[f"{prefix}_mean_FA"] = np.nan
        global_features[f"{prefix}_mean_MD"] = np.nan
        global_features[f"{prefix}_mean_GFA"] = np.nan
        global_features[f"{prefix}_FA_asymmetry"] = np.nan
        for q in range(1, 5):
            global_features[f"{prefix}_FA_Q{q}"] = np.nan
        continue

    clean_sl = Streamlines([s for s in bundle if len(s) >= 10])
    if len(clean_sl) < 10:
        continue

    # Sample FA/MD/GFA along streamlines
    profiles = {}
    for metric_name, metric_map in {"FA": FA, "MD": MD, "GFA": GFA}.items():
        try:
            profile = afq_profile(metric_map, clean_sl, affine=affine, n_points=100)
            profiles[metric_name] = profile
        except:
            profiles[metric_name] = np.full(100, np.nan)

    # Compute features
    prefix = name
    global_features[f"{prefix}_mean_FA"] = float(np.nanmean(profiles["FA"]))
    global_features[f"{prefix}_mean_MD"] = float(np.nanmean(profiles["MD"]))
    global_features[f"{prefix}_mean_GFA"] = float(np.nanmean(profiles["GFA"]))

    # FA asymmetry: (anterior - posterior) / (anterior + posterior)
    fa_profile = profiles["FA"]
    anterior = np.nanmean(fa_profile[:25])   # Q1 = start (cortical end)
    posterior = np.nanmean(fa_profile[75:])  # Q4 = end (deep or opposite side)
    asym = (anterior - posterior) / (anterior + posterior + 1e-10)
    global_features[f"{prefix}_FA_asymmetry"] = float(asym)

    # FA quartiles (Q1 = start, Q4 = end)
    for i in range(4):
        q_mean = np.nanmean(fa_profile[25*i : 25*(i+1)])
        global_features[f"{prefix}_FA_Q{i+1}"] = float(q_mean)

# Convert to DataFrame
df_final = pd.DataFrame([global_features])


# 💾 7. Save Features Set

output_path = OUTPUT_DIR / "dwi_features_clinical.csv"
df_final.to_csv(output_path, index=False)

# Print summary
n_features = df_final.shape[1]
print(f"✅ Saved {n_features} clinically meaningful features to: {output_path}")
print("\n📦 Included feature groups:")
print("  - Global: mean_FA, mean_MD, mean_GFA")
print("  - Per bundle (CC_ForcepsMajor, CC_ForcepsMinor, CST_L, CST_R):")
print("      • mean_FA, mean_MD, mean_GFA")
print("      • FA_asymmetry (anterior vs posterior)")
print("      • FA_Q1, FA_Q2, FA_Q3, FA_Q4 (profile along tract)")


# 📈 8. Visualization

print("\n" + "=" * 80)
print("STEP 8: VISUALIZATION")
print("=" * 80)

# Microstructural maps
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
mid_z = data.shape[2] // 2
plots = [
    (FA, "Fractional Anisotropy", "hot", 0, 1),
    (MD, "Mean Diffusivity (mm²/s)", "viridis", 0, 0.002),
    (GFA, "Generalized FA", "plasma", 0, 0.6),
    (np.nan_to_num(dti_fit.rd), "Radial Diffusivity (mm²/s)", "cividis", 0, 0.002),
]

for ax, (data_map, title, cmap, vmin, vmax) in zip(axes.flat, plots):
    im = ax.imshow(data_map[:, :, mid_z].T, cmap=cmap, origin="lower", vmin=vmin, vmax=vmax)
    ax.set_title(title, fontweight="bold", fontsize=11)
    ax.axis("off")
    cbar = plt.colorbar(im, ax=ax, fraction=0.04, pad=0.04)
    if "mm²/s" in title:
        cbar.ax.set_ylabel("×10⁻³", rotation=0, labelpad=15)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "microstructural_maps.png", dpi=200, bbox_inches="tight")
print("✓ Saved: microstructural_maps.png")

# Bundle profiles — FIXED: use `recognized_bundles` instead of `all_features`
n_bundles = len(recognized_bundles)
if n_bundles > 0:
    fig, axes = plt.subplots(1, min(n_bundles, 4), figsize=(4 * min(n_bundles, 4), 3))
    if min(n_bundles, 4) == 1:
        axes = [axes]
    
    for ax, (name, bundle) in zip(axes, list(recognized_bundles.items())[:4]):
        clean = Streamlines([s for s in bundle if len(s) >= 10])
        if clean:
            try:
                profile = afq_profile(FA, clean, affine=affine, n_points=100)
                ax.plot(profile, "b-", linewidth=2, alpha=0.8)
                ax.fill_between(range(100), profile, alpha=0.2, color="blue")
                ax.set_title(f"{name}", fontweight="bold")
                ax.set_ylabel("FA")
                ax.set_xlabel("Tract position (%)")
                ax.set_ylim(0, 1)
                ax.grid(True, alpha=0.3)
            except Exception as e:
                ax.text(0.5, 0.5, "Profile\nunavailable", ha="center", va="center", transform=ax.transAxes)
        else:
            ax.text(0.5, 0.5, "No data", ha="center", va="center", transform=ax.transAxes)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "bundle_profiles.png", dpi=200, bbox_inches="tight")
    print("✓ Saved: bundle_profiles.png")
else:
    print("⚠️ No bundles to visualize.")

plt.close("all")
print("\n✅ All visualizations saved!")


# 📝 Summary

print("\n" + "=" * 80)
print("PIPELINE COMPLETE: PUBLICATION-READY FEATURE EXTRACTION")
print("=" * 80)
print(f"\n📊 Final Output:")
print(f"  - Subject: Stanford HARDI")
print(f"  - Streamlines: {len(streamlines):,}")
print(f"  - Bundles extracted: {len(recognized_bundles)}")
print(f"  - Total features: {df_final.shape[1]}")
print(f"\n📁 Files generated:")
for f in OUTPUT_DIR.iterdir():
    if f.is_file():
        print(f"  - {f.name}")

✅ Imports and setup complete.
📁 Output directory: /mnt/movement/users/jaizor/xtra/notebooks/DWI/publication_dwi_features
STEP 1: DATA LOADING & QUALITY CONTROL
✓ Data shape: (81, 106, 76, 160) (x, y, z, volumes)
✓ Voxel size: (2.0, 2.0, 2.0) mm
✓ b-values: [   0. 2000.] s/mm²
✓ b=0 volumes: 10
✓ Diffusion volumes: 150

📊 Signal Quality:
  - Mean b=0 intensity: 330.9
  - Mean DWI intensity: 55.7
  - Estimated SNR: 1.61

STEP 2: PREPROCESSING

🔧 Applying Patch2Self denoising...


  ✓ Completed in 59.5s

🔧 Removing Gibbs ringing artifacts...
  ✓ Gibbs removal applied

🧠 Extracting brain mask...
  ✓ Brain voxels: 208,649 (32.0%)

STEP 3: MICROSTRUCTURAL RECONSTRUCTION
🔧 Fitting Diffusion Tensor Model...

📊 White Matter Statistics (FA > 0.2):
  FA:  0.395 ± 0.149
  MD:  0.000616 mm²/s

🔧 Estimating CSD response function...
  ✓ Response eigenvalues: [0.00128607 0.00029784 0.00029784]
  ✓ FA threshold ratio: 0.232

🔧 Fitting CSD model (sh_order=6)...

📊 Generalized FA (GFA) in WM:
  Mean: 0.831 ± 0.071

STEP 4: WHOLE-BRAIN TRACTOGRAPHY
🌱 Generated 147,066 seeds in white matter

🧵 Running probabilistic tractography...
✓ Generated 296,171 streamlines in 521.2s
✓ Retained 209,071 streamlines (20–200 mm)

STEP 5: ROI-BASED BUNDLE EXTRACTION
  ✓ CC_ForcepsMajor: 23359 streamlines
  ✓ CC_ForcepsMinor: 18000 streamlines
  ✓ CST_L: 52631 streamlines
  ✓ CST_R: 54635 streamlines

✅ Successfully extracted 4 bundles

STEP 6: EXTRACTING ROBUST, CLINICALLY RELEVANT FEATURES
✅ Sa

In [4]:
list(df_final.columns)

['subject_id',
 'global_mean_FA',
 'global_mean_MD',
 'global_mean_GFA',
 'CC_ForcepsMajor_mean_FA',
 'CC_ForcepsMajor_mean_MD',
 'CC_ForcepsMajor_mean_GFA',
 'CC_ForcepsMajor_FA_asymmetry',
 'CC_ForcepsMajor_FA_Q1',
 'CC_ForcepsMajor_FA_Q2',
 'CC_ForcepsMajor_FA_Q3',
 'CC_ForcepsMajor_FA_Q4',
 'CC_ForcepsMinor_mean_FA',
 'CC_ForcepsMinor_mean_MD',
 'CC_ForcepsMinor_mean_GFA',
 'CC_ForcepsMinor_FA_asymmetry',
 'CC_ForcepsMinor_FA_Q1',
 'CC_ForcepsMinor_FA_Q2',
 'CC_ForcepsMinor_FA_Q3',
 'CC_ForcepsMinor_FA_Q4',
 'CST_L_mean_FA',
 'CST_L_mean_MD',
 'CST_L_mean_GFA',
 'CST_L_FA_asymmetry',
 'CST_L_FA_Q1',
 'CST_L_FA_Q2',
 'CST_L_FA_Q3',
 'CST_L_FA_Q4',
 'CST_R_mean_FA',
 'CST_R_mean_MD',
 'CST_R_mean_GFA',
 'CST_R_FA_asymmetry',
 'CST_R_FA_Q1',
 'CST_R_FA_Q2',
 'CST_R_FA_Q3',
 'CST_R_FA_Q4']